In [1]:
data_path = '/data/fmri/data/'

In [2]:
from PIL import Image
from easydict import EasyDict as edict
from nilearn import image, plotting

In [3]:
import matplotlib.pyplot as plt

In [ ]:
import numpy as np
import torch


In [ ]:
def readImage(params, sub):
    sub = 'sub-hc{:03d}'.format(sub)
    device = torch.device("cuda:0" if (torch.cuda.is_available() and params.nGPU > 0) else "cpu")
    nData = params.nBacks * params.nTasks
    fAll = torch.tensor(np.zeros((nData, params.nT, params.nX, params.nY, params.nZ)), dtype = torch.float, device = device)
#     image = transforms.Normalize((0, 0, 0), (1, 1, 1))(image)

    for back in range(params.nBacks):
        for task in range(params.nTasks):
            path = params.path + 'data/' + sub + '/pb04.' + sub + '.r0' + str(task+1) +'_'+ str(2*back) +'back.scale+tlrc.nii'
            #idx = (sub * params.nBacks * params.nTasks) + (back * params.nTasks) + task
            idx =(back * params.nTasks) + task
            f = image.load_img(path)
            f = f.get_data()[:params.nX, :params.nY, :params.nZ, :params.nT]
            f = torch.tensor(f, dtype = torch.float, device = device)
            f = (f - f.mean()) / f.std() 
            fAll[idx,:,:,:,:] = f.permute(3, 0, 1, 2)
    return fAll

In [ ]:
params = edict({
    'path': data_path,
    'nGPU': 2,
    'nEpochs': 10,
    'nBacks': 2,
    'nTasks': 4,
    'nClass': 4,
    'nT': 135,
    'nX': 54,
    'nY': 64,
    'nZ': 50,
    'nDivT': 9,
    'ndf': 64,
    'lr': 0.001,
    'beta1': 0.5,
    'beta2': 0.999,
})

In [ ]:
x = readImage(params, 2)

In [ ]:
x.shape

## Visualizing Neural Images

In [ ]:
import os

In [ ]:
path = '../data/sub-hc002'
files = [os.path.join(path, f) for f in os.listdir(path) if f.endswith('.nii')]

In [ ]:
import nibabel as nib
img = nib.load(files[0])

In [ ]:
img.shape

In [ ]:
print(image.load_img(files[0]).shape)

In [ ]:
first_image = image.index_img(files[0], 0)

In [ ]:
first_image.shape

In [ ]:
plotting.plot_anat(first_image)

In [ ]:
plotting.plot_epi(first_image)

In [ ]:
plotting.plot_stat_map(first_image)

#### Looping over 4D files

In [ ]:
for img in image.iter_img(files[0]):
    plotting.plot_stat_map(img, threshold=3, display_mode='z', cut_coords=1, colorbar=False)